## Import some prerequirement

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow import keras                                                                  
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import TensorBoard 

## Set hyperparameters

In [ ]:
IMG_WIDTH = 28                                                            
IMG_HEIGHT = 28                                                           
EPOCHS = 100                                                              
BATCH_SIZE= 128                                                           
n_classes = 10                                                            

## Initialize train and test data

In [ ]:
data = np.load('mnist.npz')
[x_train, y_train, x_test, y_test] = data['x_train'], data['y_train'], data['x_test'], data['y_test']

## Show dataset

In [ ]:
plt.figure(figsize=(25,4))
for i in range(8):
    plt.subplot(1,8,i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(y_train[i])

In [ ]:
x_test  = x_test.astype(float) / 255.
x_train = x_train.astype(float) / 255.

y_test = keras.utils.to_categorical(y_test, n_classes)
y_train = keras.utils.to_categorical(y_train, n_classes)

## Define model

In [ ]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT)))     
    model.add(keras.layers.Flatten())  
    model.add(keras.layers.Dense(units=10, activation='softmax'))   
    model.add(keras.layers.Dense(units=64, activation='relu'))                           
    model.add(keras.layers.Dense(units=256, activation='relu'))
    return model

In [ ]:

%load_ext tensorboard
%tensorboard --logdir logs

## Compile and fit model

In [ ]:
RMSProp_model = build_model()        
RMSProp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
RMSProp_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/RMSProp.hdf5',monitor="val_accuracy",save_best_only=True)
RMSProp_tensorboard = TensorBoard(log_dir='logs/RMSProp')
RMSProp_callbacks = [
    RMSProp_modelcheckpoint,
    RMSProp_tensorboard]
RMSProp_history = RMSProp_model.fit(x_train, y_train,
                                    validation_data=(x_test, y_test),
                                    verbose=0,
                                    shuffle=True,
                                    epochs=EPOCHS,
                                    batch_size=BATCH_SIZE,
                                    callbacks=RMSProp_callbacks)

In [ ]:
AdaGrad_model = build_model()        
AdaGrad_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adagrad(), metrics=['accuracy'])
AdaGrad_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/AdaGrad.hdf5',monitor="val_accuracy",save_best_only=True)
AdaGrad_tensorboard = TensorBoard(log_dir='logs/AdaGrad')
AdaGrad_callbacks = [
    AdaGrad_modelcheckpoint,
    AdaGrad_tensorboard]
AdaGrad_history = AdaGrad_model.fit(x_train, y_train,
                                    validation_data=(x_test, y_test),
                                    verbose=0,
                                    shuffle=True,
                                    epochs=EPOCHS,
                                    batch_size=BATCH_SIZE,
                                    callbacks=AdaGrad_callbacks)

In [ ]:
Adam_model = build_model()       
Adam_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
Adam_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/Adam.hdf5',monitor="val_accuracy",save_best_only=True)
Adam_tensorboard = TensorBoard(log_dir='logs/Adam')
Adam_callbacks = [
    Adam_modelcheckpoint,
    Adam_tensorboard]
Adam_history = Adam_model.fit(x_train, y_train,
                              validation_data=(x_test, y_test),
                              verbose=0,
                              shuffle=True,
                              epochs=EPOCHS,
                              batch_size=BATCH_SIZE,
                              callbacks=Adam_callbacks)

## Load and compile best model

In [ ]:
Loaded_AdaGrad_model = keras.models.load_model('best_models/AdaGrad.hdf5')
Loaded_RMSProp_model = keras.models.load_model('best_models/RMSProp.hdf5')
Loaded_Adam_model = keras.models.load_model('best_models/Adam.hdf5')
Loaded_AdaGrad_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adagrad(), metrics=['accuracy'])
Loaded_RMSProp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
Loaded_Adam_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

## Evaluate best model

In [ ]:
print('RMSProp:')
print(Loaded_RMSProp_model.evaluate(x_test, y_test))
print('AdaGrad:')
print(Loaded_AdaGrad_model.evaluate(x_test, y_test))
print('Adam:')
print(Loaded_Adam_model.evaluate(x_test, y_test))

## Get Reports

In [ ]:
def evaluation(y_test, y_pred, title):
    cr = classification_report(y_test, y_pred)
    print(title + ' Result - Classification Report:')
    print(cr)
    print(title + ' Result - Confusion Matrix:')
    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, range(10), range(10))
    plt.figure(figsize = (10, 7))
    sn.set(font_scale = 1.4)
    sn.heatmap(df_cm , annot = True, annot_kws = {"size" : 16}, fmt = 'g') 
    plt.show()
    

In [ ]:
RMSProp_predict = np.argmax(Loaded_RMSProp_model.predict(x_test), axis=1)
RMSProp_expected = np.argmax(y_test, axis=1)
evaluation(RMSProp_expected, RMSProp_predict, 'RMSProp')


In [ ]:
AdaGrad_predict = np.argmax(Loaded_AdaGrad_model.predict(x_test), axis=1)
AdaGrad_expected = np.argmax(y_test, axis=1)
evaluation(AdaGrad_expected, AdaGrad_predict, 'AdaGrad')

In [ ]:
Adam_predict = np.argmax(Loaded_Adam_model.predict(x_test), axis=1)
Adam_expected = np.argmax(y_test, axis=1)
evaluation(Adam_expected, Adam_predict, 'Adam')